In [2]:
import numpy as np
import pandas as pd

import itertools

import torch
import torch.nn as nn
import torch.optim as optim

from hypll import nn as hnn
from hypll.tensors import TangentTensor
from hypll.optim import RiemannianAdam
from hypll.manifolds.poincare_ball import Curvature, PoincareBall

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [3]:
import sys
sys.path.append('../')
import util

In [8]:
AROMA = ['A_malt_all', 'A_malt_grain', 'A_malt_bread', 'A_malt_cara',
'A_malt_burn', 'A_hops_all', 'A_hops_citrus', 'A_hops_tropical',
'A_hops_noble', 'A_hops_woody', 'A_esters_all', 'A_esters_ethac',
'A_esters_isoaa', 'A_esters_flower', 'A_esters_fruity']

FLAVOUR = ['F_malt_all', 'F_malt_grain', 'F_malt_bread', 'F_malt_cara', 
'F_malt_burn', 'F_hops_all', 'F_hops_citrus', 'F_hops_tropical', 
'F_hops_noble', 'F_hops_woody', 'F_esters_all', 'F_esters_ethac',
'F_esters_isoaa', 'F_esters_flower', 'F_esters_fruity']

BOTH = ['A_malt_all', 'A_malt_grain', 'A_malt_bread', 'A_malt_cara',
'A_malt_burn', 'A_hops_all', 'A_hops_citrus', 'A_hops_tropical',
'A_hops_noble', 'A_hops_woody', 'A_esters_all', 'A_esters_ethac',
'A_esters_isoaa', 'A_esters_flower', 'A_esters_fruity', 'F_malt_all',
'F_malt_grain', 'F_malt_bread', 'F_malt_cara', 'F_malt_burn',
'F_hops_all', 'F_hops_citrus', 'F_hops_tropical', 'F_hops_noble',
'F_hops_woody', 'F_esters_all', 'F_esters_ethac', 'F_esters_isoaa',
'F_esters_flower', 'F_esters_fruity']

REST = ['acidity', 'bitternes','sweetness', 'X4vg', 'diacetyl', 'dms',
'metallic', 'stale_hops', 't2n', 'orange', 'coriander', 'clove', 'lactic',
'acetic', 'barnyard', 'alcohol', 'aftertaste', 'body', 'co2', 'overall']

ALL = ['A_malt_all', 'A_malt_grain', 'A_malt_bread', 'A_malt_cara',
'A_malt_burn', 'A_hops_all', 'A_hops_citrus', 'A_hops_tropical',
'A_hops_noble', 'A_hops_woody', 'A_esters_all', 'A_esters_ethac',
'A_esters_isoaa', 'A_esters_flower', 'A_esters_fruity', 'F_malt_all',
'F_malt_grain', 'F_malt_bread', 'F_malt_cara', 'F_malt_burn',
'F_hops_all', 'F_hops_citrus', 'F_hops_tropical', 'F_hops_noble',
'F_hops_woody', 'F_esters_all', 'F_esters_ethac', 'F_esters_isoaa',
'F_esters_flower', 'F_esters_fruity', 'acidity', 'bitternes',
'sweetness', 'X4vg', 'diacetyl', 'dms', 'metallic', 'stale_hops', 't2n',
'orange', 'coriander', 'clove', 'lactic', 'acetic', 'barnyard',
'alcohol', 'aftertaste', 'body', 'co2', 'overall']

In [10]:
LABEL_COLS = BOTH

In [11]:
df_train_X = pd.read_csv('../data/beer_features_train_samples_small.csv', index_col=0)
df_train_y = pd.read_csv('../data/beer_labels_panel_train_samples_small.csv', index_col=0)[LABEL_COLS]
df_val_X = pd.read_csv('../data/beer_features_train.csv', index_col=0)
df_val_y = pd.read_csv('../data/beer_labels_panel_train.csv', index_col=0)[LABEL_COLS]
df_test_X = pd.read_csv('../data/beer_features_test.csv', index_col=0)
df_test_y = pd.read_csv('../data/beer_labels_panel_test.csv', index_col=0)[LABEL_COLS]

train_X = df_train_X.values
train_y = df_train_y.values
val_X = df_val_X.values
val_y = df_val_y.values
test_X = df_test_X.values
test_y = df_test_y.values

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape

((1750, 231), (1750, 30), (175, 231), (175, 30), (75, 231), (75, 30))

In [12]:
df_train_y.columns

Index(['A_malt_all', 'A_malt_grain', 'A_malt_bread', 'A_malt_cara',
       'A_malt_burn', 'A_hops_all', 'A_hops_citrus', 'A_hops_tropical',
       'A_hops_noble', 'A_hops_woody', 'A_esters_all', 'A_esters_ethac',
       'A_esters_isoaa', 'A_esters_flower', 'A_esters_fruity', 'F_malt_all',
       'F_malt_grain', 'F_malt_bread', 'F_malt_cara', 'F_malt_burn',
       'F_hops_all', 'F_hops_citrus', 'F_hops_tropical', 'F_hops_noble',
       'F_hops_woody', 'F_esters_all', 'F_esters_ethac', 'F_esters_isoaa',
       'F_esters_flower', 'F_esters_fruity'],
      dtype='object')

In [13]:
df_train_y

,A_malt_all,A_malt_grain,A_malt_bread,A_malt_cara,A_malt_burn,A_hops_all,A_hops_citrus,A_hops_tropical,A_hops_noble,A_hops_woody,...,F_hops_all,F_hops_citrus,F_hops_tropical,F_hops_noble,F_hops_woody,F_esters_all,F_esters_ethac,F_esters_isoaa,F_esters_flower,F_esters_fruity
0,-0.941996,-1.075726,-0.285841,-0.178741,-0.604294,-0.656378,0.099024,0.241602,-1.268708,-0.956593,...,-0.579178,0.049896,0.394143,-0.699003,-0.489993,0.269429,0.256784,0.265336,0.252748,0.884750
1,-0.893153,-0.977069,-0.195571,-0.179653,-0.543785,-0.639002,0.101024,0.256538,-1.409950,-0.906288,...,-0.587823,0.267661,0.272445,-0.709525,-0.588568,0.408465,0.330896,0.257570,0.160407,0.917691
2,-0.880786,-1.137411,-0.336594,-0.230738,-0.601020,-0.634813,0.015769,0.156042,-1.255992,-0.990153,...,-0.532024,0.222864,0.307719,-0.707494,-0.559350,0.364755,0.260127,0.259842,0.104715,0.975348
3,-0.879230,-1.071754,-0.219354,-0.193606,-0.624828,-0.643238,0.045454,0.138254,-1.286502,-0.981138,...,-0.658764,0.168499,0.340776,-0.747876,-0.543543,0.417304,0.290938,0.220279,0.156029,0.831098
4,-0.875527,-1.063935,-0.205753,-0.175031,-0.584146,-0.621780,0.057810,0.077312,-1.368296,-0.878475,...,-0.462325,0.225635,0.333147,-0.852606,-0.565921,0.253423,0.269553,0.359259,0.203860,0.835554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,-1.233758,-0.868750,-0.638104,-0.875494,-0.278827,0.173468,1.801565,0.154312,-0.052745,-0.926968,...,-0.199043,0.393259,-0.062328,-0.198177,-0.873516,-0.439124,-0.571566,-0.152859,0.955879,-0.472166
1746,-1.063261,-0.751914,-0.690999,-0.924281,-0.199706,0.088800,1.762356,0.196654,-0.057584,-0.970705,...,-0.241538,0.380833,-0.016418,-0.088493,-0.886290,-0.455828,-0.626556,-0.211931,0.980663,-0.437777
1747,-1.153341,-0.802079,-0.600642,-0.885723,-0.224749,0.186860,1.835002,0.190688,-0.184398,-0.911978,...,-0.289067,0.469711,-0.103032,-0.149013,-0.920699,-0.412000,-0.697202,-0.144763,0.922279,-0.484735
1748,-1.203214,-0.841736,-0.702683,-0.924173,-0.245043,0.189425,1.795088,0.202355,-0.090596,-0.851933,...,-0.229928,0.327766,-0.113218,-0.158142,-0.973524,-0.497657,-0.649775,-0.144909,1.004456,-0.433727


In [14]:
FOLDS = 5
NUM_SAMPLE_TYPES = len(val_X)
NUM_SAMPLES_PER_TYPE = len(train_X) // NUM_SAMPLE_TYPES

fold_nums = list(range(FOLDS))
[num*NUM_SAMPLE_TYPES for num in fold_nums]
[(num+1)*NUM_SAMPLE_TYPES for num in fold_nums]

val_indices, train_indices = util.get_fold_indices_rand(NUM_SAMPLE_TYPES, NUM_SAMPLES_PER_TYPE, FOLDS)
train_indices
# print(FOLD_INDICES)

[array([   0,    1,    2, ..., 1747, 1748, 1749]),
 array([   0,    1,    2, ..., 1747, 1748, 1749]),
 array([  10,   11,   12, ..., 1747, 1748, 1749]),
 array([   0,    1,    2, ..., 1737, 1738, 1739]),
 array([   0,    1,    2, ..., 1747, 1748, 1749])]

In [15]:
# Define custom PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

<h1> Hyperbolic </h1>

In [16]:
# Define your MLP model
class HYP_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers, manifold):
        super(HYP_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = hnn.HLinear(input_size, layer_size, manifold=manifold)
        self.relu = hnn.HReLU(manifold=manifold)
        self.hidden_fcs = nn.ModuleList([hnn.HLinear(layer_size, layer_size, manifold=manifold) for _ in range(num_hidden_layers)])
        self.fc_out = hnn.HLinear(layer_size, output_size, manifold=manifold)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x


# Define training function
def hyp_train_model(model, train_loader, criterion, optimizer, manifold, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        tangents = TangentTensor(data=inputs, man_dim=-1, manifold=manifold)
        manifold_inputs = manifold.expmap(tangents)

        outputs = model(manifold_inputs)

        loss = criterion(outputs.tensor, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

<h1> EUCLIDEAN </h1>

In [17]:
# Define your MLP model
class EUC_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers):
        super(EUC_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.hidden_fcs = nn.ModuleList([nn.Linear(layer_size, layer_size) for _ in range(num_hidden_layers)])
        self.fc_out = nn.Linear(layer_size, output_size)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x

# Define training function
def euc_train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

In [22]:
# param_grid = {
#     'model_type': ['hyp', 'euc'],
#     'num_hidden_layers': [0,1,2,4,8],
#     'layer_size': [2,4,8,16,32,64,128],
#     'lr': [0.005,0.01,0.02],
#     'weight_decay': [0.005,0.01,0.02],
#     'batch_size': [1024],
#     'epochs': [40],
#     'curvature': [-1]
# }

param_grid = {
    'model_type': ['hyp', 'euc'],
    'num_hidden_layers': [0,1,2,4,8],
    'layer_size': [2,4,8,16,32,64,128,256,512,1024,1536,2048],
    'lr': [0.005,0.01,0.02,0.03,0.04],
    'weight_decay': [0.005,0.01,0.02,0.03,0.04],
    'batch_size': [1024],
    'epochs': [40],
    'curvature': [-1]
}

param_combinations = list(itertools.product(*param_grid.values()))
len(param_combinations)

3000

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_eval_stats = []

for i, params in enumerate(param_combinations):
    print(f'----- Combination {i} -----')
    print(*zip(param_grid.keys(), params))
    model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = params

    for fold, (fold_train_indices, fold_val_indices) in enumerate(zip(train_indices, val_indices)):
        print(f'Fold {fold}')

        fold_train_X = train_X[fold_train_indices]
        fold_train_y = train_y[fold_train_indices]
        fold_val_X   = val_X[fold_val_indices]
        fold_val_y   = val_y[fold_val_indices]
        # fold_val_X   = test_X
        # fold_val_y   = test_y

        train_dataset = CustomDataset(fold_train_X, fold_train_y)
        val_dataset = CustomDataset(fold_val_X, fold_val_y)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        if model_type == 'hyp':
            manifold = PoincareBall(c=Curvature(curvature))
        elif model_type == 'euc':
            manifold = None

        if model_type == 'hyp':
            model = HYP_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers,
                            manifold=manifold).to(device)
        elif model_type == 'euc':
            model = EUC_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers).to(device)

        criterion = nn.MSELoss()

        if model_type == 'hyp':
            optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
        elif model_type == 'euc':
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

        for epoch in range(epochs):
            if model_type == 'hyp':
                eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
                eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

                eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
                eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
            elif model_type == 'euc':
                eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
                eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

                eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
                eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

        print(eval_stats['mae']['val'])
        param_eval_stats.append(eval_stats)

----- Combination 0 -----
('model_type', 'hyp') ('num_hidden_layers', 0) ('layer_size', 2) ('lr', 0.005) ('weight_decay', 0.005) ('batch_size', 1024) ('epochs', 40) ('curvature', -1)
Fold 0


<h1>BEST MODEL</h1>

In [12]:
best_params = {
    'model_type': 'hyp',
    'num_hidden_layers': 4,
    'layer_size': 256,
    'lr': 0.003,
    'weight_decay': 0.001,
    'batch_size': 1024,
    'epochs': 10,
    'curvature': -1
}

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_eval_stats = []

model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = best_params.values()

for fold in [0]:
    print(f'Fold {fold}')

    fold_train_X = train_X
    fold_train_y = train_y
    fold_val_X   = test_X
    fold_val_y   = test_y

    train_dataset = CustomDataset(fold_train_X, fold_train_y)
    val_dataset = CustomDataset(fold_val_X, fold_val_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    if model_type == 'hyp':
        manifold = PoincareBall(c=Curvature(curvature))
    elif model_type == 'euc':
        manifold = None

    if model_type == 'hyp':
        model = HYP_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers,
                        manifold=manifold).to(device)
    elif model_type == 'euc':
        model = EUC_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers).to(device)

    criterion = nn.MSELoss()

    if model_type == 'hyp':
        optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif model_type == 'euc':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

    for epoch in range(epochs):
        if model_type == 'hyp':
            eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
            eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

            eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
            eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
        elif model_type == 'euc':
            eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
            eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

            eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
            eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

    print(eval_stats['mae']['val'])
    param_eval_stats.append(eval_stats)

Fold 0
[0.015503318091556252, 0.046809887485494105, 0.06806168484875695, 0.08266557620391923, 0.09373429069746794, 0.10036593844714145, 0.10486519088700184, 0.1087505170615147, 0.11201089459243274, 0.11489738420749455]
